# Packet Analysis Using Data Science

***

## Getting Started

***

The first thing we are going to do in order to make all this possible, is downloading and importing a few libraries. These include:
+ [**Scapy**](https://scapy.net)- For packet manupilation.
+ [**Pandas**](https://pandas.pydata.org/) - To help us create and manupilate dataframes.
+ [**Numpy**](http://www.numpy.org/) - To help us perform complex mathematical functions.
+ [**Binascii**](https://docs.python.org/2/library/binascii.html) - To help us convert from Binary to Ascii.
+ [**Seaborn**](https://seaborn.pydata.org/) - For some awesome visualization. 

We are also going to add the Matplotlib inline (`%matplotlib inline`) function, to allow any visualization to appear within the notebook itself. 

In [3]:
!pip install scapy

    100% |████████████████████████████████| 798kB 1.3MB/s eta 0:00:01
  Running setup.py bdist_wheel for scapy ... error
  Complete output from command /opt/conda/envs/DSX-Python35/bin/python -u -c "import setuptools, tokenize;__file__='/home/dsxuser/.tmp/pip-build-m05jwmzb/scapy/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /home/dsxuser/.tmp/tmpdzmxdv3npip-wheel- --python-tag cp35:
  /opt/conda/envs/DSX-Python35/lib/python3.5/distutils/dist.py:261: UserWarning: Unknown distribution option: 'project_urls'
    warnings.warn(msg)
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib
  creating build/lib/scapy
  copying scapy/dadict.py -> build/lib/scapy
  copying scapy/route6.py -> build/lib/scapy
  copying scapy/main.py -> build/lib/scapy
  copying scapy/asn1fields.py -> build/lib/scapy
  copying scapy/route.py -> build/lib/scapy
  copying

In [4]:
%matplotlib inline

from scapy.all import *
import pandas as pd
import numpy as np
import binascii
import seaborn as sns

sns.set(color_codes=True)

## Sniffing packets in Scapy

***

Scapy is a tool that provides powerfull and interactive packet manupilation. it allows you to forge or decode packets of a wide number of protocols, send them on wire, capture them, match requests and replies, etc. Packet capture and analysis can primarily be accomplished using Wireshark, however, it is hard to keep track of multiple suspicious indicators while also keeping track of multiple connections in Wireshark. Manupliating packets in Scapy can get a little rigid, so we transform our packet capture into a Panda DataFrame. 



In [8]:
# num_of_packets_to_sniff = 100
# pcap = sniff(count=num_of_packets_to_sniff)

# print(type(pcap))
# print(len(pcap))
# print(pcap)
# pcap[0]

PermissionError: [Errno 1] Operation not permitted

In [ ]:
pcap = pcap + 